## Introduction

In recent years there has been an explosion in the amount of available data, due to the extensive availability of low-cost computer memory and automated data collection systems. The regression modeling techniques of the past were developed to work well under ideal theoretical conditions provided basic assumptions are met. In practice however, they typically don't work well with large data sets (observations in the thousands and feature space 50+ dimensions). In the modern era, it is not uncommon to be faced with data sets involving millions of observations and hundreds or thousands of predictors. This exponential growth in available data has motivated researchers in the' fields of statistics, artificial intelligence, and data mining to develop a few brand new procedures for data modeling that can be applied to very large data sets. However, it turns out that flexible, powerful state of the art methods were discovered from theoretical standpoint decades (and even centuries, in the case of Bayesian updating) ago. the missing element has always been the ability of computers to efficiently execute these algorithms in user-friendly ways. The one technique that most exemplifies this narrative is the neural network.

Neural networks are patterned after the system of neurons and synapses in the human brain. Neurons define the process our brains use to react to stimuli around us. The following paragraph does a good job of explaining the process:

*Neurons are typically classified into three types based on their function. [Sensory neurons](https://en.wikipedia.org/wiki/Sensory_neuron "Sensory neuron") respond to [stimuli](https://en.wikipedia.org/wiki/Stimulus_(physiology) "Stimulus (physiology)") such as touch, sound, or light that affect the cells of the [sensory organs](https://en.wikipedia.org/wiki/Sense "Sense"), and they send signals to the spinal cord or brain. [Motor neurons](https://en.wikipedia.org/wiki/Motor_neuron "Motor neuron") receive signals from the brain and spinal cord to control everything from [muscle contractions](https://en.wikipedia.org/wiki/Muscle_contraction "Muscle contraction") to [glandular output](https://en.wikipedia.org/wiki/Gland "Gland"). [Interneurons](https://en.wikipedia.org/wiki/Interneuron "Interneuron") connect neurons to other neurons within the same region of the brain or spinal cord. When multiple neurons are functionally connected together, they form what is called a [neural circuit](https://en.wikipedia.org/wiki/Neural_circuit "Neural circuit")* (retrieved from <https://en.wikipedia.org/wiki/Neuron>).

![](images/paste-D3518732.png)

Image from:<https://www.biolegend.com/de-at/synaptic-function>

It is important to note that these "functional connections" result in unique architectures. For example the visual information received through our eyes is processed differently from the audio information processed through our ears.

## Modeling with Neural Networks


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sklearn'

The basic idea behind the neural network approach in regression and classification problems is to model the response as a *nonlinear* function of various [linear combinations]{.underline} of the predictors (Independent variables/inputs). One common aspect of every neural network is that it consists of an **input layer**, whose dimension is based on the nature of the features (predictors, independent, explanatory variables) in our data. Similarly, the dimension of the **output** layer is based on the target variable (response, label) we seek to predict.

![](images/paste-37781E17.png){width="478"}

Diagram Created with [https://alexlenail.me/NN-SVG/index.html](https://alexlenail.me/NN-SVG/index.html%20https://alexlenail.me/NN-SVG/index.html)

The image depicts a *single layer perceptron* neural network architecture. It is called a [feed forward]{.underline} network because data enters the network from the left and moves through the network until it exits on the right with a predicted value for our response. It is the most fundamental type of neural network we can work with. Let's discuss the components:

The Input Layer consists of 5 neurons. 4 corresponding to the features in our data, and one corresponding to the intercept, or *bias* term we are choosing to include in our model.

The synapses connecting the input layer to the output layer are known as weights. These are unknown parameters that must be learned through our data. [If input variables are on an equal scale]{.underline}, then the values if the weights relative to each other give an indication as to which are more influential in predicting the outcome variable. Keeping variables on the same scale corrects over/under emphasizing variables simply based on order of magnitude. There are a number of pre-processing steps that can be used to put variables on a common scale. This step is common to many machine learning models, not just neural networks.

Standardization

$X^*=\frac{X_{ij}-mean(X_j)}{stdev(X_j)}$

MAX-MIN Normalization 

$X^*=\frac{X_{ij}-min(X_j)}{max(X_j)-min(X_j)}$

This is the method of choice for neural networks, as it typicaly makes for a higher likelihood of model stability stability when the weights are being learned.

The neurons of the first layer form a linear combination through the weights to create the constant $Z_i=\beta_{0} + \beta_{1}X_{i1} +...+ \beta_{4}X_{i4}$ for individual $i$.

The *activation function* $g$ may be interpreted based on whether a regression or classification problem is being done. In the regression context, we can simply see $g$ as a function used to model nonlinearity in the relationship between predictors and response. In the classification context, consider the biological context. When would an audio stimulus cause you to cover your ears? Analogously, in the model we could consider the activation potential being met if the value of $Z_i$ exceeds 0, leading to a positive classification, and not otherwise. In other words, our classification rule is

$$
\hat{Y}_i=
    \begin{cases}        1 & \text{if } Z_i \geq 0\\        
                         0 & \text{if } Z_i < 0    
    \end{cases}
$$

The drawback to this rule is that it leads to a non-smooth function as the output of our network. We will soon find out that a critical part of training our model will be determining the location in the parameter space that minimizes some loss criterion. This means we'll be taking derivatives, and possibly logarithms, so smoothness is a must. Thus we prefer activation functions with the same range from 0 to 1, but also continuously hitting all values in between. Popular choices are seen in the picture below.

![](paste-1830B82C.png)

How should one choose the "best" activation function? Unlike classical statistical methods where model assumptions dictate what is appropriate, in machine learning oftentimes it is the nature of the data, and performance that determine which action to take. In March 2023, researchers at MIT discovered a new type of activation function that seems to [outperform the standard options](https://news.mit.edu/2023/method-designing-neural-networks-optimally-suited-certain-tasks-0330). According to lead author, Adityanarayanan Radhakrishnan, "It turns out that, if you take the standard activation functions that people use in practice, and keep increasing the depth of the network, it gives you really terrible performance. We show that if you design with different activation functions, as you get more data, your network will get better and better." The article was published in the [Proceedings of the Natural Academy of Sciences](https://www.pnas.org/doi/10.1073/pnas.2208779120).

It is a methodical interplay between the quantities defined above and the data to learn the values of the weights that are in some sense optimal.

**Exercise 1: The standard multiple regression model involves just one linear combination of the predictors, namely:**

$$ \mu_i=E\{Y_{i}\} = \beta_{0} + \beta_{1}X_{i1} +...+ \beta_{p-1}X_{i,p-1} $$

**How would a neural network that mirrors this scenario look? Express directly and in vector/matrix notation. For a given data set, do you think the weights would be the same as the parameter estimates obtained via least squares?**

**Exercise 2: The standard logistic regression model involves just one linear combination of the predictors, namely:**

$$ \pi_i=E\{Y_{i}\} = \frac{1}{1+\exp\left[-(\beta_{0} + \beta_{1}X_{i1} +...+ \beta_{p-1}X_{i,p-1})\right]} $$

**How would a neural network that mirrors this scenario look? Express directly and in vector/matrix notation. For a given data set, do you think the weights would be the same as the parameter estimates obtained via maximum likelihood?**

Despite the similarities, the neural network model (in particular deep learning models - where there are additional layers between the input and output) is simply a nonlinear statistical model that contains many more parameters than the corresponding linear statistical model. One result of this is that the models will typically be overparameterized, resulting in parameters that are uninterpretable, which is a major shortcoming of neural network modeling. An advantage of the neural network approach is that the resulting model will often perform better in predicting future responses than a standard regression model.

## Deep Learning

The single layer perceptron is powerful in itself as a generalization of familiar statistical models. But that doesn't begin to scratch the surface on the massive array of customized architectures of arbitrary complexity that can be created in the neural net framework. It is important to note that model complexity and flexibility are not merely theoretical novelties. The theoretical properties of the neural net have allowed it to be applied to a wide variety of classical and modern problems in machine learning and artificial intelligence, with success metrics that often rival the human brains they were modeled after. All this while using high-powered computing. This being said, **it is important to recognize that the nature of the input and output layers discussed above remain the same**. The input layer will be dictated by the feature space, and the output layer by the problem we are tackling: regression, classification, etc. We must apply familiar principles of model training, testing and hyperparameter tuning to improve architecture towards optimality.

![](images/paste-0D341572.png){width="494"}

Example: General idea of Deep Learning

![](images/paste-2CC859C1.png)Example: A deep NN with (4,6,5,7,3) Architecture

The good news is that for a wide array of common tasks, this work has been done repeatedly with simulated and real data sets, and with computing power orders of magnitude greater than what we can do with consumer machines. The concept of *Transfer Learning* is the study of how to effectively apply pre-trained models to harness their predictive power without needing to start from scratch. This topic will be discussed in further detail later this semester.

**Example: Building a three-layer network for regression modelling.**


In [ ]:
data = load_diabetes()
X = data['data']
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 7)
        self.fc2 = nn.Linear(7, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
net = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# Train the model
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch} loss: {loss.item()}")

# Test the model
with torch.no_grad():
    y_pred = net(X_test)
    test_loss = criterion(y_pred, y_test)
    print(f"Test loss: {test_loss.item()}")

In [ ]:
# Define a new input to make predictions on
new_input = [[0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0]]

# Standardize the input using the same scaler we used on the training data
new_input = scaler.transform(new_input)

# Convert the input to a PyTorch tensor
new_input = torch.tensor(new_input, dtype=torch.float32)

# Make a prediction using the trained model
with torch.no_grad():
    prediction = net(new_input)
    print(f"Prediction: {prediction.item()}")

### Custom Architectures for Specific Learning Challenges

#### Computer Vision - Convolutional Neural Networks

Convolutional Neural Networks (CNNs) are a specific type of neural network that are particularly well-suited for processing image and video data. They are based on the concept of convolution, which involves sliding a filter over the input image and computing a dot product between the filter weights and the corresponding pixel values at each location. The formula for the convolution operation is:

$(f * w)(i, j) = ΣΣ f(m, n) * w(i-m, j-n)$

where:

-   f is the input image

-   w is the kernel

-   i and j are the indices of the output feature map

-   m and n are the indices of the kernel

![](https://lh4.googleusercontent.com/8h8GN8pIRQAYb0pmxeMQAr-BK9g-RwSYDg5s7xPGf5MlafAPpQDTA4esBy11Ze-wevSgVxlRk9ud9IPfwFmUgojxEvowwFwc8YuX5vZrJsTG_Qb0_Dg2017hP3WWDRogqiUBbEdW7w4d)

CNNs are typically composed of multiple layers, including convolutional layers, pooling layers, and fully connected layers. In a convolutional layer, the network "learns" to extract features from the input image by convolving it with a set of learned filters. The resulting feature maps capture different aspects of the image, such as edges, corners, and textures. Pooling layers are used to downsample the feature maps, reducing their dimensionality and making them more computationally efficient. This is typically done by taking the maximum or average value over a local region of the feature map. The formula for max pooling is:

$y(i,j) = max { f(i',j') : i' ∈ [istride,istride+p), j' ∈ [jstride,jstride+p) }$

where:

-   f is the input feature map

-   y is the output pooled feature map

-   i and j are the indices of the output feature map

-   p is the pooling window size

-   stride is the stride of the pooling operation

 Finally, fully connected layers are used to combine the extracted features and produce the final output. These layers are similar to those used in traditional neural networks, but they operate on the flattened feature maps rather than the raw input data.

During training, the weights of the CNN are adjusted using backpropagation and stochastic gradient descent, similar to traditional neural networks. However, because of the specific structure of CNNs, they can learn to extract high-level features from images and capture complex relationships between them.

#### Sequential Data - Recurrent Neural Networks

#### Text Data - Transformer Neural Networks

## Training the Weights: Backpropogation and Gradient Descent